# Analyze Stored Wilcoxon Test Results

This notebook loads a previously saved `EEGAnalyzer` object and displays the significant (p &lt; 0.05) results from the stored Wilcoxon signed-rank tests.

In [1]:
import pandas as pd
import os
from eeg_analyzer.eeg_analyzer import EEGAnalyzer

# Configuration
analyzer_name_to_load = "UnprocessedAnalyzer" # Replace with your analyzer's name if different
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

## Load EEGAnalyzer Object

In [2]:
analyzer = EEGAnalyzer.load_analyzer(analyzer_name_to_load)

if analyzer:
    print(f"Successfully loaded EEGAnalyzer: {analyzer.analyzer_name}")
    print(analyzer)
else:
    print(f"Failed to load EEGAnalyzer: {analyzer_name_to_load}")

[EEGAnalyzer - UnprocessedAnalyzer] EEGAnalyzer state loaded from /home/sivert/Documents/Master_AttentionalDirectionResearch/data/eeg_analyzer_derivatives/UnprocessedAnalyzer/analyzer_state.pkl
Successfully loaded EEGAnalyzer: UnprocessedAnalyzer
--- EEGAnalyzer Instance: 'UnprocessedAnalyzer' ---
  Description: Unprocessed data
  Datasets (3): braboszcz2017, jin2019, touryan2022
  DataFrame Status: DataFrame with shape (1996288, 14)
  Derivatives Path: /home/sivert/Documents/Master_AttentionalDirectionResearch/data/eeg_analyzer_derivatives/UnprocessedAnalyzer
--------------------------------------


## Display Significant Wilcoxon Test Results

Iterate through all stored statistical analyses, identify Wilcoxon test results, and display significant findings.

In [3]:
if analyzer and hasattr(analyzer, 'statistical_analyses') and analyzer.statistical_analyses:
    print(f"\n--- Stored Statistical Analyses in '{analyzer.analyzer_name}' ---")
    
    for analysis_name, analysis_content in analyzer.statistical_analyses.items():
        # We are interested in analyses that look like our Wilcoxon results
        if "wilcoxon_meanOT_vs_meanMW" in analysis_name:
            print(f"\n\n======================================================")
            print(f"Analysis: {analysis_name}")
            print(f"======================================================")
            
            print("\nParameters used for this analysis:")
            if analysis_content.get('parameters'):
                for param, value in analysis_content['parameters'].items():
                    print(f"  - {param}: {value}")
            else:
                print("  - No parameters recorded.")
            
            print(f"\nTimestamp of analysis: {analysis_content.get('timestamp')}")

            significance_data = analysis_content.get('data')
            if not significance_data:
                print("\nNo data found for this analysis.")
                continue

            print("\n--- Significant Results (p < 0.05) ---")
            found_significant_results_for_analysis = False
            for tail, datasets_results in significance_data.items():
                print(f"\n  Tail: '{tail}'")
                found_significant_for_tail = False
                for dataset_name, results_df in datasets_results.items():
                    if results_df is not None and not results_df.empty:
                        # Ensure p_value column exists
                        if 'p_value' in results_df.columns:
                            significant_df = results_df[results_df['p_value'] < 0.05]
                            if not significant_df.empty:
                                print(f"\n    Dataset: {dataset_name}")
                                print(significant_df.to_string())
                                found_significant_for_tail = True
                                found_significant_results_for_analysis = True
                            else:
                                pass # print(f"    Dataset: {dataset_name} - No significant results for p < 0.05.")
                        else:
                            print(f"    Dataset: {dataset_name} - 'p_value' column missing in results DataFrame.")
                    # else:
                        # print(f"    Dataset: {dataset_name} - No results data or empty DataFrame.")
                if not found_significant_for_tail:
                    print(f"    No significant results (p < 0.05) found for any dataset under tail '{tail}'.")
            
            if not found_significant_results_for_analysis:
                 print(f"\nNo significant results (p < 0.05) found for this analysis: {analysis_name}")
        else:
            print(f"\nSkipping analysis '{analysis_name}' as it does not appear to be a Wilcoxon test of interest.")
            
else:
    if analyzer:
        print("\nNo statistical analyses found in the loaded analyzer object.")
    else:
        print("Analyzer object not loaded, cannot display results.")


--- Stored Statistical Analyses in 'UnprocessedAnalyzer' ---


Analysis: wilcoxon_meanOT_vs_meanMW_by_channel

Parameters used for this analysis:
  - col1: mean_OT
  - col2: mean_MW
  - group_cols: ['channel']
  - n_permutations: 9999
  - random_state: 42
  - source_csv_file_name: summary_subses_channel_level_unprocessed_band_db_statecomp.csv

Timestamp of analysis: 2025-05-30 23:30:24.166448

--- Significant Results (p < 0.05) ---

  Tail: 'greater'

    Dataset: jin2019
   channel  wilcoxon_statistic  p_value
0      AF3              1126.0   0.0205
1      AF4              1156.0   0.0106
2      AF7              1138.0   0.0171
3      AF8              1220.0   0.0028
4      AFz              1119.0   0.0216
9       C5              1146.0   0.0127
10      C6              1208.0   0.0031
12     CP2              1102.0   0.0274
14     CP4              1169.0   0.0083
15     CP5              1162.0   0.0105
16     CP6              1192.0   0.0045
19      F1              1132.0   0.0152
20

---
End of Analysis
---